In [1]:
import cirq
import numpy as np

# Function to generate a random bit string
def random_bits(n):
    return np.random.randint(2, size=n) 

# Prepare qubits and encode them based on Alice's random bits and bases
def prepare_qubits(bit_string, basis_string, qubits):
    for i, bit in enumerate(bit_string):
        if basis_string[i] == 0:  # Prepare in computational basis
            if bit == 1:
                yield cirq.X(qubits[i])
        else:  # Prepare in Hadamard basis
            if bit == 1:
                yield cirq.X(qubits[i])
            yield cirq.H(qubits[i])

# Bob's measurement in random bases
def measure_in_basis(qubits, basis_string):
    for i, basis in enumerate(basis_string):
        if basis == 1:
            yield cirq.H(qubits[i])
        yield cirq.measure(qubits[i], key=f'q{i}')

def QKD_simulator(total_qubits):

    # Step 1: Alice generates random bits and bases
    alice_bits = random_bits(total_qubits)
    alice_bases = random_bits(total_qubits)

    # Step 2: Bob generates random bases
    bob_bases = random_bits(total_qubits)

    # Step 3: Prepare qubits based on Alice's bits and bases
    qubits = [cirq.LineQubit(i) for i in range(total_qubits)]
    circuit = cirq.Circuit()
    circuit += prepare_qubits(alice_bits, alice_bases, qubits)
    print('Alice_qubits', circuit)

    # Step 4: Bob measures the qubits in his bases
    circuit += measure_in_basis(qubits, bob_bases)
    print('bob measurement', circuit)

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1)

    # Extract Bob's measurement results
    bob_results = [int(result.measurements[f'q{i}'][0]) for i in range(total_qubits)]

    # Step 5: Generate the shared key
    shared_key = []
    for i in range(total_qubits):
        if alice_bases[i] == bob_bases[i]:
            shared_key.append(bob_results[i])

    print("Alice's bits:   ", alice_bits)
    print("Alice's bases:  ", alice_bases)
    print("Bob's bases:    ", bob_bases)
    print("Bob's results:  ", bob_results)
    print("Shared key:     ", shared_key)

In [2]:
QKD_simulator(10)

Alice_qubits 0: ───X───H───

1: ───X───────

3: ───H───────

5: ───H───────

6: ───X───────

7: ───H───────

8: ───X───────

9: ───H───────
bob measurement 0: ───X─────────H─────────M('q0')───

1: ───X─────────M('q1')─────────────

2: ───M('q2')───────────────────────

3: ───H─────────H─────────M('q3')───

4: ───H─────────M('q4')─────────────

5: ───H─────────M('q5')─────────────

6: ───X─────────H─────────M('q6')───

7: ───H─────────M('q7')─────────────

8: ───X─────────H─────────M('q8')───

9: ───H─────────H─────────M('q9')───
Alice's bits:    [1 1 0 0 0 0 1 0 1 0]
Alice's bases:   [1 0 0 1 0 1 0 1 0 1]
Bob's bases:     [0 0 0 1 1 0 1 0 1 1]
Bob's results:   [1, 1, 0, 0, 0, 0, 0, 0, 1, 0]
Shared key:      [1, 0, 0, 0]
